# Parallel questions study
In this notebook, we carry out the study of the parallel questions related to the influence of movies on baby names, therefore conduction a global analysis.

In [16]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from scipy import stats
import seaborn as sns

In [17]:
folder_processed_data_path = './data/processed_data/'
movie_df = pd.read_csv(os.path.join(folder_processed_data_path, 'movie_df.csv'))
movie_df.set_index(['wiki_ID'], inplace=True)
name_by_movie = pd.read_csv(os.path.join(folder_processed_data_path, 'name_by_movie_ordered_pvalue_10_5_df.csv'))
name_by_movie.set_index(['wiki_ID'], inplace=True)
display(movie_df)
display(name_by_movie)

,mov_name,year,month,revenue,numVotes,averageRating
wiki_ID,,,,,,
975900,Ghosts of Mars,2001,8.0,14010832.0,56880,4.9
3196793,Getting Away with Murder: The JonBenét Ramsey ...,2000,2.0,NaN,69,6.0
28463795,Brun bitter,1988,NaN,NaN,40,5.6
9363483,White Of The Eye,1987,NaN,NaN,2891,6.1
261236,A Woman in Flames,1983,NaN,NaN,623,5.9
...,...,...,...,...,...,...
35228177,Mermaids: The Body Found,2011,3.0,NaN,1711,4.6
34980460,Knuckle,2011,1.0,NaN,3192,6.8
9971909,Another Nice Mess,1972,9.0,NaN,111,5.8


,char_words,order,gender,t_stat,p_value
wiki_ID,,,,,
3217,Gold,6.0,NaN,NaN,NaN
3217,Linda,7.0,F,0.429187,0.676072
3217,Henry,4.0,M,2.019954,0.068422
3217,Duke,4.0,M,-0.566260,0.582585
3217,Warrior,9.0,M,NaN,NaN
...,...,...,...,...,...
37478048,Ajay,9.0,M,0.806658,0.436957
37501922,Murphy,3.0,F,-1.257988,0.234444
37501922,Hunter,1.0,M,7.051709,0.000021


### Question 1: Month of release

In [18]:
name_by_movie_aggregate_df = name_by_movie.merge(movie_df, how='left', left_on='wiki_ID', right_on='wiki_ID')
display(summer_movies_df)

,char_words,order,gender,t_stat,p_value,mov_name,year,month,revenue,numVotes,averageRating
wiki_ID,,,,,,,,,,,
3217,Gold,6.0,NaN,NaN,NaN,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Linda,7.0,F,0.429187,0.676072,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Henry,4.0,M,2.019954,0.068422,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Duke,4.0,M,-0.566260,0.582585,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Warrior,9.0,M,NaN,NaN,Army of Darkness,1992,10.0,21502796.0,191068,7.4
...,...,...,...,...,...,...,...,...,...,...,...
37478048,Ajay,9.0,M,0.806658,0.436957,Mr. Bechara,1996,NaN,NaN,395,5.4
37501922,Murphy,3.0,F,-1.257988,0.234444,Terminal Bliss,1992,NaN,NaN,245,4.4
37501922,Hunter,1.0,M,7.051709,0.000021,Terminal Bliss,1992,NaN,NaN,245,4.4


In [32]:
summer = [6.0, 7.0, 8.0]
fall = [9.0,10.0,11.0]
winter = [12.0,1.0,2.0]
spring = [3.0,4.0,5.0]
summer_movies_df = name_by_movie_aggregate_df[name_by_movie_aggregate_df['month'].isin(summer)]
fall_movies_df = name_by_movie_aggregate_df[name_by_movie_aggregate_df['month'].isin(fall)]
winter_movies_df = name_by_movie_aggregate_df[name_by_movie_aggregate_df['month'].isin(winter)]
spring_movies_df = name_by_movie_aggregate_df[name_by_movie_aggregate_df['month'].isin(spring)]


display(summer_movies_df)
display(fall_movies_df)
display(winter_movies_df)
display(spring_movies_df)

,char_words,order,gender,t_stat,p_value,mov_name,year,month,revenue,numVotes,averageRating
wiki_ID,,,,,,,,,,,
3746,Deckard,0.0,M,NaN,NaN,Blade Runner,1982,6.0,33139618.0,804384,8.1
3746,Eldon,8.0,M,0.460773,0.653938,Blade Runner,1982,6.0,33139618.0,804384,8.1
3746,Lewis,12.0,M,1.024419,0.327638,Blade Runner,1982,6.0,33139618.0,804384,8.1
3746,Bear,11.0,M,NaN,NaN,Blade Runner,1982,6.0,33139618.0,804384,8.1
3746,Leon,7.0,M,-0.748319,0.469966,Blade Runner,1982,6.0,33139618.0,804384,8.1
...,...,...,...,...,...,...,...,...,...,...,...
36699915,Luke,5.0,M,-0.198023,0.846639,Percy Jackson & the Olympians: Sea of Monsters,2013,8.0,NaN,123248,5.7
36699915,Underwood,1.0,M,NaN,NaN,Percy Jackson & the Olympians: Sea of Monsters,2013,8.0,NaN,123248,5.7
36699915,Chase,2.0,F,-1.551732,0.149005,Percy Jackson & the Olympians: Sea of Monsters,2013,8.0,NaN,123248,5.7


,char_words,order,gender,t_stat,p_value,mov_name,year,month,revenue,numVotes,averageRating
wiki_ID,,,,,,,,,,,
3217,Gold,6.0,NaN,NaN,NaN,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Linda,7.0,F,0.429187,0.676072,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Henry,4.0,M,2.019954,0.068422,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Duke,4.0,M,-0.566260,0.582585,Army of Darkness,1992,10.0,21502796.0,191068,7.4
3217,Warrior,9.0,M,NaN,NaN,Army of Darkness,1992,10.0,21502796.0,191068,7.4
...,...,...,...,...,...,...,...,...,...,...,...
37322106,Major,0.0,M,1.927686,0.080101,Jab Tak Hai Jaan,2012,11.0,NaN,58012,6.7
37373877,Beth,5.0,F,0.826799,0.425923,Crazy Eights,2006,10.0,NaN,3338,3.8
37373877,Patterson,5.0,F,NaN,NaN,Crazy Eights,2006,10.0,NaN,3338,3.8


,char_words,order,gender,t_stat,p_value,mov_name,year,month,revenue,numVotes,averageRating
wiki_ID,,,,,,,,,,,
3837,Lamarr,3.0,M,-0.305851,0.765429,Blazing Saddles,1974,2.0,119500000.0,147934,7.7
3837,Van,11.0,M,1.216756,0.249160,Blazing Saddles,1974,2.0,119500000.0,147934,7.7
3837,Bart,0.0,M,-0.179090,0.861123,Blazing Saddles,1974,2.0,119500000.0,147934,7.7
3837,Lyle,6.0,M,0.155542,0.879211,Blazing Saddles,1974,2.0,119500000.0,147934,7.7
3837,Buddy,18.0,M,-0.043728,0.965905,Blazing Saddles,1974,2.0,119500000.0,147934,7.7
...,...,...,...,...,...,...,...,...,...,...,...
36956792,Kid,18.0,M,NaN,NaN,The Water Horse: Legend of the Deep,2007,12.0,103071443.0,42523,6.4
36956792,Charlie,5.0,M,5.427450,0.000208,The Water Horse: Legend of the Deep,2007,12.0,103071443.0,42523,6.4
36956792,Beach,18.0,M,NaN,NaN,The Water Horse: Legend of the Deep,2007,12.0,103071443.0,42523,6.4


,char_words,order,gender,t_stat,p_value,mov_name,year,month,revenue,numVotes,averageRating
wiki_ID,,,,,,,,,,,
4560,Morrison,19.0,M,NaN,NaN,Braveheart,1995,5.0,211409945.0,1072580,8.3
4560,Edward,3.0,M,0.377036,0.713319,Braveheart,1995,5.0,211409945.0,1072580,8.3
4560,Campbell,5.0,M,1.721204,0.113180,Braveheart,1995,5.0,211409945.0,1072580,8.3
4560,Murron,1.0,F,NaN,NaN,Braveheart,1995,5.0,211409945.0,1072580,8.3
4560,William,0.0,M,3.384440,0.006095,Braveheart,1995,5.0,211409945.0,1072580,8.3
...,...,...,...,...,...,...,...,...,...,...,...
36814246,Girl,4.0,F,NaN,NaN,Eraserhead,1977,3.0,7000000.0,124128,7.3
36814246,Mary,1.0,F,2.799750,0.017282,Eraserhead,1977,3.0,7000000.0,124128,7.3
36814246,Beautiful,4.0,F,NaN,NaN,Eraserhead,1977,3.0,7000000.0,124128,7.3


In [29]:
summer_movies_df.index.unique()

Index([    3746,     3947,     4231,     4726,     4727,     4728,     4729,
           4730,     8481,     9979,
       ...
       36306987, 36329343, 36354051, 36354224, 36422681, 36448415, 36478252,
       36566804, 36617100, 36699915],
      dtype='int64', name='wiki_ID', length=4096)

In [50]:
prop_summer = len(summer_movies_df[summer_movies_df['p_value']<0.1])/len(summer_movies_df['p_value'])
display(prop_summer)
prop_fall = len(fall_movies_df[fall_movies_df['p_value']<0.1])/len(fall_movies_df['p_value'])
display(prop_fall)
prop_winter = len(winter_movies_df[winter_movies_df['p_value']<0.1])/len(winter_movies_df['p_value'])
display(prop_winter)
prop_spring = len(spring_movies_df[spring_movies_df['p_value']<0.1])/len(spring_movies_df['p_value'])
display(prop_spring)

0.13600104190407972

0.14431685722080306

0.13875639832480224

0.13644032637256256

Faire test statistique pour dire que different ?